In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score


%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}
pd.options.mode.chained_assignment = None  # default='warn'?

# data_key DataFrame
data_key = pd.read_csv('key.csv')

# data_train DataFrame
data_train = pd.read_csv('train.csv')

# data_weather DataFrame
data_weather = pd.read_csv('weather.csv')

rain_text = ['FC', 'TS', 'GR', 'RA', 'DZ', 'SN', 'SG', 'GS', 'PL', 'IC', 'FG', 'BR', 'UP', 'FG+']
other_text = ['HZ', 'FU', 'VA', 'DU', 'DS', 'PO', 'SA', 'SS', 'PY', 'SQ', 'DR', 'SH', 'FZ', 'MI', 'PR', 'BC', 'BL', 'VC' ]

data_weather['codesum'].replace("+", "")
a = []
for i in range(len(data_weather['codesum'])):
    a.append(data_weather['codesum'].values[i].split(" "))
    for i_text in a[i]:
        if len(i_text) == 4:
            a[i].append(i_text[:2])
            a[i].append(i_text[2:])
            
data_weather["nothing"] = 1
data_weather["rain"] = 0
data_weather["other"] = 0
b = -1
for ls in a:
    b += 1
    for text in ls:
        if text in rain_text:
            data_weather.loc[b, 'rain'] = 1
            data_weather.loc[b, 'nothing'] = 0
        elif text in other_text:
            data_weather.loc[b,'other'] = 1
            data_weather.loc[b, 'nothing'] = 0      
# 모든 데이터 Merge
df = pd.merge(data_weather, data_key)

station_nbr = df['station_nbr']
df.drop('station_nbr', axis=1, inplace=True)
df['station_nbr'] = station_nbr

df = pd.merge(df, data_train)

# T 값 처리 하기. Remained Subject = > 'M' and '-'
df['snowfall'][df['snowfall'] == '  T'] = 0.05
df['preciptotal'][df['preciptotal'] == '  T'] = 0.005

# 주말과 주중 구분 작업 하기
df['date'] = pd.to_datetime(df['date'])
df['week7'] = df['date'].dt.dayofweek
df['weekend'] = 0
df.loc[df['week7'] == 5, 'weekend'] = 1
df.loc[df['week7'] == 6, 'weekend'] = 1

df1 = df[df['station_nbr'] == 1]; df11 = df[df['station_nbr'] == 11]
df2 = df[df['station_nbr'] == 2]; df12 = df[df['station_nbr'] == 12]
df3 = df[df['station_nbr'] == 3]; df13 = df[df['station_nbr'] == 13]
df4 = df[df['station_nbr'] == 4]; df14 = df[df['station_nbr'] == 14]
df5 = df[df['station_nbr'] == 5]; df15 = df[df['station_nbr'] == 15]
df6 = df[df['station_nbr'] == 6]; df16 = df[df['station_nbr'] == 16]
df7 = df[df['station_nbr'] == 7]; df17 = df[df['station_nbr'] == 17]
df8 = df[df['station_nbr'] == 8]; df18 = df[df['station_nbr'] == 18]
df9 = df[df['station_nbr'] == 9]; df19 = df[df['station_nbr'] == 19]
df10 = df[df['station_nbr'] == 10]; df20 = df[df['station_nbr'] == 20]

In [5]:
df16 = df16.apply(pd.to_numeric, errors = 'coerce')
df16.describe().iloc[:, 13:]
# 없는 Column = date, codesum, depart, sunrise, sunset, station_nbr, snowfall

,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed,nothing,rain,other,store_nbr,station_nbr,item_nbr,units,week7,weekend
count,210900.000000,210900.000000,209568.000000,209790.000000,209790.000000,209568.000000,211344.000000,211344.000000,211344.000000,211344.000000,211344.0,211344.000000,211344.000000,211344.000000,211344.000000
mean,0.089511,29.872821,29.982214,5.908995,21.096296,7.297458,0.480042,0.516807,0.071429,29.500000,16.0,56.000000,0.231102,2.998950,0.285714
std,0.243714,0.228264,0.229363,3.479431,9.782755,3.262658,0.499603,0.499719,0.257540,15.500037,0.0,32.041715,2.327981,2.001317,0.451755
min,0.000000,28.970000,29.160000,0.300000,1.000000,0.500000,0.000000,0.000000,0.000000,14.000000,16.0,1.000000,0.000000,0.000000,0.000000
25%,0.000000,29.720000,29.830000,3.200000,14.000000,5.000000,0.000000,0.000000,0.000000,14.000000,16.0,28.000000,0.000000,1.000000,0.000000
50%,0.005000,29.870000,29.990000,5.400000,23.000000,6.800000,0.000000,1.000000,0.000000,29.500000,16.0,56.000000,0.000000,3.000000,0.000000
75%,0.040000,30.030000,30.140000,7.900000,29.000000,8.900000,1.000000,1.000000,0.000000,45.000000,16.0,84.000000,0.000000,5.000000,1.000000
max,2.650000,30.610000,30.730000,21.700000,36.000000,23.400000,1.000000,1.000000,1.000000,45.000000,16.0,111.000000,113.000000,6.000000,1.000000


In [2]:
df16['store_nbr'].unique()

array([14, 45], dtype=int64)

In [6]:
df16_drop_columns = ['date', 'codesum', 'depart', 'sunrise', 'sunset', 'station_nbr', 'snowfall']
df16 = df16.drop(columns = df16_drop_columns)

In [7]:
# np.nan를 포함하고 있는 변수(column)를 찾아서, 그 변수에 mean 값 대입해서 Frame의 모든 Value가 fill 되게 하기.
df16_columns = df16.columns

# Cateogry 값을 포함하는 변수는 np.nan에 mode값으로 대체하고, 나머지 실수 값을 포함한 변수는 np.nan에 mean값으로 대체
for i in df16_columns:
    if (i == 'resultdir'):
        df16[i].fillna(df16[i].mode()[0], inplace=True)
        print(df16[i].mode()[0])
    else:
        df16[i].fillna(df16[i].mean(), inplace=True)

# 이제 모든 변수가 숫자로 표기 되었기 때문에, 가능 함. 
# 상대 습도 추가 #
df16['relative_humility'] = 100*(np.exp((17.625*((df16['dewpoint']-32)/1.8))/(243.04+((df16['dewpoint']-32)/1.8)))/np.exp((17.625*((df16['tavg']-32)/1.8))/(243.04+((df16['tavg']-32)/1.8))))

# 체감온도 계산
df16["windchill"] = 35.74 + 0.6216*df16["tavg"] - 35.75*(df16["avgspeed"]**0.16) + 0.4275*df16["tavg"]*(df16["avgspeed"]**0.16)

df16 = df16[df16['units'] != 0]

31.0


In [8]:
# 'np.log1p(units) ~ tmax + tmin + tavg + depart + dewpoint + wetbulb + heat + cool + sunrise + sunset + codesume + \
# snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir + avgspeed + nothing + rain + other + \
# store_nbr + station_nbr + item_nbr'

model_df16 = sm.OLS.from_formula('np.log1p(units) ~ tmax + tmin + tavg + dewpoint + heat + cool + preciptotal + \
resultspeed + sealevel + resultdir + avgspeed + C(nothing) + C(rain) + C(other) + C(item_nbr) + C(week7) + \
C(weekend) + relative_humility + windchill + 0', data = df16)
result_df16 = model_df16.fit()

print(result_df16.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.465
Model:                            OLS   Adj. R-squared:                  0.461
Method:                 Least Squares   F-statistic:                     126.0
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:51:09   Log-Likelihood:                -4858.8
No. Observations:                4671   AIC:                             9784.
Df Residuals:                    4638   BIC:                             9996.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
C(nothing)[0]         0.0205      0.10

In [9]:
model_df16 = sm.OLS.from_formula('np.log1p(units) ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(heat) + scale(cool) + \
scale(preciptotal) + scale(resultspeed) + scale(sealevel) + scale(resultdir) + scale(avgspeed) + C(nothing) + C(rain) + \
C(other) + C(item_nbr) + C(week7) + C(weekend) + scale(relative_humility) + scale(windchill) + 0', data = df16)
result_df16 = model_df16.fit()

print(result_df16.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.465
Model:                            OLS   Adj. R-squared:                  0.461
Method:                 Least Squares   F-statistic:                     126.0
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:51:23   Log-Likelihood:                -4858.8
No. Observations:                4671   AIC:                             9784.
Df Residuals:                    4638   BIC:                             9996.
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(nothing)[0]           

In [10]:
anova_result_df16 = sm.stats.anova_lm(result_df16).sort_values(by=['PR(>F)'], ascending = False)
anova_result_df16[anova_result_df16['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
scale(resultspeed),1.0,1.965237,1.965237,4.161903,4.139947e-02
scale(heat),1.0,2.237570,2.237570,4.738638,2.954268e-02
scale(tavg),1.0,2.618803,2.618803,5.545999,1.856442e-02
scale(tmin),1.0,3.673930,3.673930,7.780505,5.302854e-03
scale(avgspeed),1.0,5.453818,5.453818,11.549883,6.832444e-04
scale(tmax),1.0,5.553575,5.553575,11.761145,6.100718e-04
C(week7),6.0,29.810176,4.968363,10.521805,1.251625e-11
C(item_nbr),11.0,1850.560493,168.232772,356.276808,0.000000e+00
C(nothing),2.0,18675.112489,9337.556244,19774.712707,0.000000e+00


In [11]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(df16.values, i) for i in range(df16.shape[1])]
vif["features"] = df16.columns
vif = vif.sort_values("VIF Factor").reset_index(drop=True)
vif
# 10순위까지 겹치는것만 쓴다
# resultspeed, avgspeed, C(week7), C(item_nbr),

,VIF Factor,features
0,1.135416,store_nbr
1,1.155370,units
2,1.176225,other
3,1.250637,item_nbr
4,1.455097,preciptotal
5,1.694285,resultdir
6,2.711633,week7
7,2.732272,weekend
8,8.001323,resultspeed
9,14.701744,avgspeed


In [13]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
model_df16 = sm.OLS.from_formula('np.log1p(units) ~ scale(resultspeed) + scale(avgspeed) + C(week7) + C(item_nbr)+ 0', data = df16)
result_df16 = model_df16.fit()

print(result_df16.summary())

                            OLS Regression Results                            
Dep. Variable:        np.log1p(units)   R-squared:                       0.460
Model:                            OLS   Adj. R-squared:                  0.458
Method:                 Least Squares   F-statistic:                     208.8
Date:                Wed, 04 Jul 2018   Prob (F-statistic):               0.00
Time:                        16:53:08   Log-Likelihood:                -4879.8
No. Observations:                4671   AIC:                             9800.
Df Residuals:                    4651   BIC:                             9928.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(week7)[0]            0.9967      0

In [14]:
# resultspeed, avgspeed, C(weekend), preciptotal, C(other), C(week7), C(item_nbr),
X16 = df16[['resultspeed', 'avgspeed', 'week7', 'item_nbr']]
y16 = df16['units']
model16 = LinearRegression()

cv16 = KFold(n_splits=16, shuffle=True, random_state=0)

cross_val_score(model16, X16, y16, scoring="r2", cv=cv16)

array([0.10276912, 0.09335096, 0.10227192, 0.06427252, 0.10270704,
       0.10017133, 0.09186556, 0.077997  , 0.08586437, 0.07850477,
       0.0462939 , 0.10844841, 0.05555789, 0.13442299, 0.08544858,
       0.08305209])